<a href="https://colab.research.google.com/github/bhanvimenghani/Transfer_learning/blob/master/vgg_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.applications import VGG16

# VGG16 was designed to work on 224 x 224 pixel input images sizes
img_rows = 224
img_cols = 224 

#Loads the VGG16 model 
model = VGG16(weights = 'imagenet',           #VGG takes weights from imagenet
                 include_top = False,         #We remove the top layer(output layer) of VGG as we will make it according to our requirement
                 input_shape = (img_rows, img_cols, 3))

Using TensorFlow backend.


58892288/58889256 [==============================] - 5s 0us/step


In [2]:
for (i,layer) in enumerate(model.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 Conv2D True
2 Conv2D True
3 MaxPooling2D True
4 Conv2D True
5 Conv2D True
6 MaxPooling2D True
7 Conv2D True
8 Conv2D True
9 Conv2D True
10 MaxPooling2D True
11 Conv2D True
12 Conv2D True
13 Conv2D True
14 MaxPooling2D True
15 Conv2D True
16 Conv2D True
17 Conv2D True
18 MaxPooling2D True


In [3]:
for layer in model.layers:
    layer.trainable = False

In [4]:
for (i,layer) in enumerate(model.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 MaxPooling2D False
11 Conv2D False
12 Conv2D False
13 Conv2D False
14 MaxPooling2D False
15 Conv2D False
16 Conv2D False
17 Conv2D False
18 MaxPooling2D False


In [5]:
def addTopModel(bottom_model, num_classes, D=256):       #D is dimensions
    """Creates the top or head of the model that will be 
    placed ontop of the bottom layers"""
    top_model = bottom_model.output
    top_model = Flatten(name = "flatten")(top_model)       #Way of adding layers ; Efficient way
    top_model = Dense(D, activation = "relu")(top_model)
    top_model = Dropout(0.3)(top_model)                    #Dropout fn is for the change of 2 that comes after convolution
    top_model = Dense(num_classes, activation = "softmax")(top_model)
    return top_model

In [6]:
model.input

<tf.Tensor 'input_1:0' shape=(None, 224, 224, 3) dtype=float32>

In [7]:
model.layers

In [8]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

num_classes = 2               #For 2 types of faces

FC_Head = addTopModel(model, num_classes)

modelnew = Model(inputs=model.input, outputs=FC_Head)    #Compile

print(modelnew.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [9]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = '/content/drive/My Drive/Train'
validation_data_dir = '/content/drive/My Drive/Test'

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 16
val_batchsize = 10
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=1,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=1,
        class_mode='categorical',
        shuffle=False)

Found 200 images belonging to 2 classes.
Found 94 images belonging to 2 classes.


In [10]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping
                   
checkpoint = ModelCheckpoint("faces_VGG16.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# Note we use a very small learning rate 
modelnew.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

#nb_train_samples = 1190
#nb_validation_samples = 170
epochs = 10
batch_size = 16

history = modelnew.fit_generator(
    train_generator,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator)

Epoch 1/10
200/200 [==============================] - 293s 1s/step - loss: 2.8581 - accuracy: 0.7100 - val_loss: 3.9964 - val_accuracy: 0.8723

Epoch 00001: val_loss improved from inf to 3.99642, saving model to faces_VGG16.h5
Epoch 2/10
200/200 [==============================] - 151s 755ms/step - loss: 0.9698 - accuracy: 0.8100 - val_loss: 1.5477 - val_accuracy: 0.8723

Epoch 00002: val_loss improved from 3.99642 to 1.54770, saving model to faces_VGG16.h5
Epoch 3/10
200/200 [==============================] - 148s 740ms/step - loss: 0.8087 - accuracy: 0.8500 - val_loss: 3.6965 - val_accuracy: 0.8830

Epoch 00003: val_loss did not improve from 1.54770
Epoch 4/10
200/200 [==============================] - 148s 738ms/step - loss: 0.5914 - accuracy: 0.8550 - val_loss: 0.1266 - val_accuracy: 0.8404

Epoch 00004: val_loss improved from 1.54770 to 0.12660, saving model to faces_VGG16.h5
Epoch 5/10
200/200 [==============================] - 148s 738ms/step - loss: 0.5467 - accuracy: 0.8850 - v

In [11]:
train_generator.class_indices

{'bhan': 0, 'mom': 1}

In [12]:
# checking the model

In [13]:
#Predicting an image

from keras.preprocessing import image
test_image = image.load_img('/content/drive/My Drive/Test/bhan/image13.jpg', target_size=(224,224))

import numpy as np
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
result = modelnew.predict(test_image)
result

array([[1.0847601e-32, 1.0000000e+00]], dtype=float32)

In [14]:
if result[0][0] == 0 :
    print("Hello Mummyji")
else :
    print("Hello Bhanvi")

Hello Bhanvi
